# Section 1
*PLACEHOLDER*

In [6]:
# Install dependencies as needed:
# Install dependencies based on OS
import os
import sys
if sys.platform == "win32":
    %pip install kagglehub[pandas-datasets]
else:
    %pip install "kagglehub[pandas-datasets]"


import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_paths = ["high_popularity_spotify_data.csv", "low_popularity_spotify_data.csv"]
database = []

# Load the latest version
for file_path in file_paths:
    df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "solomonameh/spotify-music-dataset",
    file_path,
    # Provide any additional arguments like 
    # sql_query or pandas_kwargs. See the 
    # documenation for more information:
    # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
    )
    database.append(df)

print("First 5 records:", df.head())

Note: you may need to restart the kernel to use updated packages.


/tmp/ipykernel_40406/1354516349.py:20: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 713k/713k [00:00<00:00, 1.32MB/s]
/tmp/ipykernel_40406/1354516349.py:20: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 1.30M/1.30M [00:00<00:00, 1.81MB/s]

First 5 records:    time_signature  track_popularity  speechiness  danceability  playlist_name  \
0             4.0                23       0.0393         0.636  Rock Classics   
1             4.0                53       0.0317         0.572  Rock Classics   
2             4.0                55       0.0454         0.591  Rock Classics   
3             4.0                64       0.1010         0.443  Jazz Classics   
4             4.0                62       0.0298         0.685  Jazz Classics   

                       track_artist  duration_ms  energy playlist_genre  \
0      Creedence Clearwater Revival     138053.0   0.746           rock   
1                         Van Halen     241600.0   0.835           rock   
2                      Stevie Nicks     329413.0   0.804           rock   
3  Ella Fitzgerald, Louis Armstrong     185160.0   0.104           jazz   
4                    Galt MacDermot     205720.0   0.472           jazz   

  playlist_subgenre  ...          track_album